## Building A Chatbot
In this We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.


In [3]:
## Importing neccessary Libraries
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from pprint import pprint

In [4]:
## Calling our LLM model
groq_api_key = os.getenv('GROQ_API_KEY')

model = ChatGroq(model='llama3-8b-8192',api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000163822B8520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000163822B9180>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
## Printing a basic response message
response = model.invoke(input='Hi my name is shiva')

pprint(response.content)

'Nice to meet you, Shiva! What brings you here today?'


In [10]:
## Using Langchains messaging function to guide LLM about messages 
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content='Hi my name is shiva')])
pprint(response)

AIMessage(content='Nice to meet you, Shiva!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 16, 'total_tokens': 25, 'completion_time': 0.0075, 'prompt_time': 0.001669932, 'queue_time': 0.011962776, 'total_time': 0.009169932}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-f9d6873a-e0cd-404c-912d-dbdd03d76ba9-0', usage_metadata={'input_tokens': 16, 'output_tokens': 9, 'total_tokens': 25})


In [11]:
## Manually providing the history of messages to LLM
from langchain_core.messages import HumanMessage, AIMessage

response = model.invoke([HumanMessage(content='Hi my name is shiva')
                ,AIMessage(content = """Nice to meet you, Shiva! What brings you here today? Do you have a particular topic 
                           you'd like to chat about or ask me something? I'm all ears (or in this case, all text)!"""),
                            HumanMessage(content="Can you tell me whats my name?") ])

response.content

"I'd be happy to remind you! Your name is Shiva!"

### Lets try to store the Chat history 

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def message_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_msg_history = RunnableWithMessageHistory(model,message_history)



In [13]:
config = {'configurable':{'session_id':'chat1'}}

with_msg_history.invoke(input="Hi my name is shiv",config=config)


AIMessage(content="Nice to meet you, Shiv! How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 16, 'total_tokens': 32, 'completion_time': 0.013333333, 'prompt_time': 0.002182966, 'queue_time': 0.344948009, 'total_time': 0.015516299}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-cf474771-0961-4ffa-a7cf-bc9299518eb0-0', usage_metadata={'input_tokens': 16, 'output_tokens': 16, 'total_tokens': 32})

In [14]:
with_msg_history.invoke(input="can you only tell me what is full name MSD from cricket?",config=config)


AIMessage(content='MSD stands for Mahendra Singh Dhoni', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 54, 'total_tokens': 64, 'completion_time': 0.008333333, 'prompt_time': 0.006381894, 'queue_time': 0.007908635, 'total_time': 0.014715227}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-56801556-82f9-4f73-b43a-fdc93c2d04c3-0', usage_metadata={'input_tokens': 54, 'output_tokens': 10, 'total_tokens': 64})

In [15]:
with_msg_history.invoke(input="can you only tell me what is my name and how do you know it ?",config=config)


AIMessage(content='Your name is Shiv, and I know it because you told me when you introduced yourself at the beginning of our conversation!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 89, 'total_tokens': 114, 'completion_time': 0.020833333, 'prompt_time': 0.011006306, 'queue_time': 0.004364632, 'total_time': 0.031839639}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-255e613c-d3bc-4a1e-b772-46106fd33d34-0', usage_metadata={'input_tokens': 89, 'output_tokens': 25, 'total_tokens': 114})

In [16]:
print(store)

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi my name is shiv', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you, Shiv! How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 16, 'total_tokens': 32, 'completion_time': 0.013333333, 'prompt_time': 0.002182966, 'queue_time': 0.344948009, 'total_time': 0.015516299}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-cf474771-0961-4ffa-a7cf-bc9299518eb0-0', usage_metadata={'input_tokens': 16, 'output_tokens': 16, 'total_tokens': 32}), HumanMessage(content='can you only tell me what is full name MSD from cricket?', additional_kwargs={}, response_metadata={}), AIMessage(content='MSD stands for Mahendra Singh Dhoni', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 54, 'total_tokens': 64, 'com

In [18]:
message_history('chat1').messages

[HumanMessage(content='Hi my name is shiv', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Nice to meet you, Shiv! How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 16, 'total_tokens': 32, 'completion_time': 0.013333333, 'prompt_time': 0.002182966, 'queue_time': 0.344948009, 'total_time': 0.015516299}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-cf474771-0961-4ffa-a7cf-bc9299518eb0-0', usage_metadata={'input_tokens': 16, 'output_tokens': 16, 'total_tokens': 32}),
 HumanMessage(content='can you only tell me what is full name MSD from cricket?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='MSD stands for Mahendra Singh Dhoni', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 54, 'total_tokens': 64, 'completion_time': 0.008333333, 'prompt_time': 

##### if we change the session id LLM will not be able to remember the context  

In [19]:
config = {'configurable':{'session_id':'chat2'}}

with_msg_history.invoke(input="can you only tell me what is my name?",config=config)


AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the capability to know your personal information, including your name. I'm designed to provide general information and assist with tasks, but I don't have access to personal data.\n\nIf you'd like to share your name with me, I'd be happy to chat with you and get to know you better!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 20, 'total_tokens': 100, 'completion_time': 0.066666667, 'prompt_time': 0.002117999, 'queue_time': 0.016166611, 'total_time': 0.068784666}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-cc51692b-f06c-4518-8b01-b8e796e4faf6-0', usage_metadata={'input_tokens': 20, 'output_tokens': 80, 'total_tokens': 100})

### Lets try using Prompt template

In [21]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
    ('system',"You are helpful assistanat please ans the question in the best of your knowledge"),
    MessagesPlaceholder(variable_name='messages') ]
)

chain = prompt | model

chain.invoke({'messages':[HumanMessage("Hi my name is shiv")]})

AIMessage(content="Nice to meet you, Shiv! It's great to have you here. Is there something I can help you with or would you like to chat about something in particular? I'm all ears (or rather, all text)!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 37, 'total_tokens': 83, 'completion_time': 0.038333333, 'prompt_time': 0.004722809, 'queue_time': 0.01031488, 'total_time': 0.043056142}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-13b80921-a07b-49c7-b863-05902f8734c5-0', usage_metadata={'input_tokens': 37, 'output_tokens': 46, 'total_tokens': 83})

In [27]:
with_msg_history = RunnableWithMessageHistory(chain, message_history)
with_msg_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function message_history at 0x0000016382201510>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [28]:
config = {"configurable": {"session_id": "chat3"}}

with_msg_history.invoke(
    {'messages': [HumanMessage('Hi my name is shiv')]}, config=config
)

AIMessage(content="I know, Shiv! You've told me already!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 311, 'total_tokens': 323, 'completion_time': 0.010304242, 'prompt_time': 0.061858839, 'queue_time': 0.0019329639999999954, 'total_time': 0.072163081}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-27f9ca0d-d0f1-4a5e-ba95-cb7e24a1a3d5-0', usage_metadata={'input_tokens': 311, 'output_tokens': 12, 'total_tokens': 323})

In [23]:
with_msg_history.invoke(
    {'messages': [HumanMessage('can you tell me where is mumbai? in the world')]}, config=config
)

AIMessage(content='Hello Shiv!\n\nMumbai is a city located on the western coast of India, in the state of Maharashtra. It is situated on a peninsula protruding into the Arabian Sea, and is the financial and entertainment capital of India.\n\nIn terms of geographical coordinates, Mumbai is located at:\n\n* Latitude: 18.9750° N\n* Longitude: 72.8258° E\n\nMumbai is a major metropolitan city and one of the most populous cities in the world, with a population of over 12.4 million people. It is known for its vibrant culture, stunning beaches, iconic landmarks like the Gateway of India and the Taj Mahal Palace Hotel, and its bustling streets that are filled with street food, markets, and fashion.\n\nI hope that helps, Shiv!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 92, 'total_tokens': 250, 'completion_time': 0.131666667, 'prompt_time': 0.011418839, 'queue_time': 0.0031720299999999993, 'total_time': 0.143085506}, 'model_name': 'llama

In [24]:
with_msg_history.invoke(
    {'messages': [HumanMessage('can you tell me what is my name?')]}, config=config
)

AIMessage(content='Shiv, your name is Shiv!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 268, 'total_tokens': 277, 'completion_time': 0.0075, 'prompt_time': 0.033353218, 'queue_time': -0.0008212919999999943, 'total_time': 0.040853218}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-6d0673dc-d7c0-47dd-af7e-54504172774a-0', usage_metadata={'input_tokens': 268, 'output_tokens': 9, 'total_tokens': 277})

In [25]:
message_history('chat3').messages

[HumanMessage(content='Hi my name is shiv', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Nice to meet you, Shiv! I'm your helpful assistant, here to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 37, 'total_tokens': 71, 'completion_time': 0.028333333, 'prompt_time': 0.004811849, 'queue_time': 0.008257502, 'total_time': 0.033145182}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7b8dbb47-0c65-4110-b9b3-123aca313e4f-0', usage_metadata={'input_tokens': 37, 'output_tokens': 34, 'total_tokens': 71}),
 HumanMessage(content='can you tell me where is mumbai? in the world', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Shiv!\n\nMumbai is a city located on the western coast of India, in the state of Maharashtra. It is situated on a peninsula pro

### Lets add some more complexity with place holders

In [29]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
    ('system',"You are helpful assistanat please ans the question in the besst of your knowledge in the following {language}"),
    MessagesPlaceholder(variable_name='messages') ]
)

chain = prompt | model

chain.invoke({'messages':[HumanMessage("Hi my name is shiv")],'language':'hindi'})

AIMessage(content='नमस्ते शिव! मैं तुम्हारा मदद करने के लिए यहाँ हूँ! क्या मैं आपके लिए कुछ कर सकता हूँ?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 42, 'total_tokens': 87, 'completion_time': 0.0375, 'prompt_time': 0.004779201, 'queue_time': 0.009307279, 'total_time': 0.042279201}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-23fe55d8-d7f5-470b-9958-f2eb7a46fb4e-0', usage_metadata={'input_tokens': 42, 'output_tokens': 45, 'total_tokens': 87})

In [30]:
with_msg_history = RunnableWithMessageHistory(chain, message_history,input_messages_key="messages")
config = {"configurable": {"session_id": "chat4"}}

with_msg_history.invoke(
    {'messages': [HumanMessage('Hi my name is shiv')],'language':'hindi'}, config=config
)

AIMessage(content='नमस्ते शिव! मैं आपकी मदद करने के लिए यहाँ हूँ! क्या आपके पास कोई सवाल है या आप कुछ जानते हुए हैं?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 42, 'total_tokens': 93, 'completion_time': 0.0425, 'prompt_time': 0.004872009, 'queue_time': 0.0093373, 'total_time': 0.047372009}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-6eebc57f-0076-443d-b6d6-62471e98536e-0', usage_metadata={'input_tokens': 42, 'output_tokens': 51, 'total_tokens': 93})

### Managing the Conversation History


In [65]:
from langchain_core.messages import SystemMessage,trim_messages

## Trimmer basis token count
trimmer = trim_messages(
    max_tokens = 40,
    strategy = 'last',
    include_system = True,
    token_counter = model,
    allow_partial = False,
    start_on = 'human'
)

messages = [
    SystemMessage(content="you're a good assistant, please answer the question based on the context if you have"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage('Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage("Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    HumanMessage(content="having fun?"),

]
pprint(trimmer.invoke(messages))


[SystemMessage(content="you're a good assistant, please answer the question based on the context if you have", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={})]


In [68]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


chain = RunnablePassthrough.assign(messages = itemgetter('messages') | trimmer) | prompt | model


response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I'm not sure! I'm a helpful assistant, but I don't have any information about your personal preferences. Can you tell me about your favorite ice cream flavors?"

In [64]:
## When i increase the max_token from 40 to 120 we see LLM picking the context

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What icecream do i like")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream!'

In [36]:
from langchain_core.messages import SystemMessage,trim_messages

## Trimmer basis message count
trimmer = trim_messages(
    max_tokens = 5,
    strategy = 'last',
    include_system = True,
    token_counter = len,
    allow_partial = False,
    start_on = 'human'
)

messages = [
    SystemMessage(content="you're a good assistant, you always respond with a joke."),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage('Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage("Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
pprint(trimmer.invoke(messages))


[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]
